In [1]:
from utils.syntax import PREPOSITIONS
import re

In [10]:
# Retain preposition tag
edit_tag = r'(\[-(.+?)-\]|\{\+(.+?)\+\})'
def retain_target(line):
    is_target = False
    matches = re.findall(edit_tag, line)
    for edit, delete, insert in matches:
        if delete in PREPOSITIONS or insert in PREPOSITIONS: is_target = True
        elif delete: line = line.replace(edit, '')
        elif insert: line = line.replace(edit, insert)
    return line if is_target else None


# Transform format [-for-] {+in+} >> [-for>>in+}
replace_format = r'(\[-([^-]+?)-\] \{\+([^+]+?)\+\})'
def transform_replace(line):
    matches = re.findall(replace_format, line)
    for edit, delete, insert in matches:
        line = line.replace(edit, "[-{}>>{}+}}".format(delete, insert))
    return line


# One line one error
new_edit_tag = r'(\[-(.+?)-\]|{\+(.+?)\+}|\[-(.+?)>>(.+?)\+})'
def duplicate(line):
    dups = []
    matches = re.findall(new_edit_tag, line)
    for i in range(len(matches)):
        dup = line
        for j, (edit, delete, insert, rd, ri) in enumerate(matches):
            if   i == j: continue
            elif delete: dup = dup.replace(edit, '')
            elif insert: dup = dup.replace(edit, insert)
            else:        dup = dup.replace(edit, ri)
        dups.append(dup)
    return dups

In [ ]:
lines = open('../dataset/wikiedit.txt', 'r', encoding='utf8', errors='ignore').readlines()

remain_lines = []
for line in lines:
    line = retain_target(line.strip())
    if line:
        line = transform_replace(line.replace('  ', ' '))
        dups = duplicate(line)
        remain_lines.extend(dups)

In [15]:
# ws = open('wikedit_prep', 'w', encoding='utf8')
# for line in remain_lines:
#     print(line, file=ws)
# ws.close()

In [19]:
short_lines = list(filter(lambda line: len(line.split(' ')) <= 25, remain_lines))

In [21]:
short_lines[:10]

['The film ends with plants growing inspite {+of+} predictions that there would be no food for sixty years .',
 'Grandi experimented with extreme emotionalism [-with>>in+} some of his music , with chromaticism , ornament and affectation',
 'The temple was constructed [-on>>by+} the orders of Nyuserre Ini , the sixth king of the fifth dynasty of Egypt .',
 'The Palais {+under+} the stewardship of Louis Philippe II .',
 'López was [-on-] one of the most highly touted players coming out of high school .',
 'He was also a member of the team just missing out of a medal in 4th-place [-at>>in+} the 2004 team competition .',
 'Cathays station is now manned during peak hours , following the introduction of a new automated ticket barrier system in Summer [-of-] 2007 .',
 'Takizawa is the largest village in Japan by size {+of+} population .',
 'It is located [-in>>at+} 1-chome , Nagatacho , Chiyoda Ward , Tokyo .',
 'Smaller sites , apparently more residential in character , are scattered around 